In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
import uuid

# Initialize Faker
fake = Faker()

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# ======================
# CONFIGURATION
# ======================
NUM_SHIPMENTS = 100000
NUM_ULDS = 5000
NUM_FLIGHTS = 2000
NUM_CUSTOMERS = 500
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2025, 1, 31)

# Airports (focus on QR cargo hubs and major destinations)
AIRPORTS = ['DOH', 'LHR', 'JFK', 'HKG', 'NRT', 'SIN', 'CDG', 'FRA', 'DXB', 'BKK', 
            'MEL', 'SYD', 'LAX', 'ORD', 'MAD', 'AMS', 'IST', 'PVG', 'CAN', 'DEL']

ULD_TYPES = ['AKE', 'AKH', 'AAP', 'AMA', 'DQF', 'PAG', 'PLA', 'PMC', 'P1P', 'P6P']
ULD_TYPE_CAPACITIES = {
    'AKE': {'max_weight': 1588, 'max_volume': 10.4},
    'AKH': {'max_weight': 1588, 'max_volume': 11.5},
    'AAP': {'max_weight': 6032, 'max_volume': 16.0},
    'AMA': {'max_weight': 6804, 'max_volume': 18.0},
    'DQF': {'max_weight': 2449, 'max_volume': 7.0},
    'PAG': {'max_weight': 6804, 'max_volume': 18.0},
    'PLA': {'max_weight': 4627, 'max_volume': 12.0},
    'PMC': {'max_weight': 6804, 'max_volume': 18.0},
    'P1P': {'max_weight': 6804, 'max_volume': 18.0},
    'P6P': {'max_weight': 13608, 'max_volume': 36.0}
}

CARGO_CATEGORIES = ['General Cargo', 'Perishables', 'Pharma', 'Valuables', 
                    'Dangerous Goods', 'Live Animals', 'E-commerce', 'Express']
CUSTOMER_SEGMENTS = ['Freight Forwarder', 'Direct Shipper', 'E-commerce Platform', 
                     'Pharma Company', 'Perishables Exporter', 'Government']
CONTRACT_TYPES = ['Spot Rate', 'Blocked Space', 'Volume Commitment', 'Long-term']

AIRCRAFT_TYPES = ['B777F', 'B747-8F', 'B777-300ER', 'A350F', 'A330-300']

# ======================
# 1. GENERATE CUSTOMER DIMENSION
# ======================
print("Generating customer dimension...")
customers = []
for i in range(NUM_CUSTOMERS):
    customer_id = f"CUST{str(i+1).zfill(6)}"
    customers.append({
        'customer_id': customer_id,
        'customer_name': fake.company(),
        'country': fake.country(),
        'customer_segment': random.choice(CUSTOMER_SEGMENTS),
        'contract_type': random.choice(CONTRACT_TYPES),
        'contract_start_date': fake.date_between(start_date=START_DATE, end_date=END_DATE - timedelta(days=180)),
        'credit_rating': random.choice(['A', 'B', 'C', 'D']),
        'yearly_spend_usd': np.random.lognormal(mean=10, sigma=1.5),
        'preferred_commodity': random.choice(CARGO_CATEGORIES)
    })

df_customers = pd.DataFrame(customers)

# ======================
# 2. GENERATE ULD MASTER
# ======================
print("Generating ULD master...")
ulds = []
for i in range(NUM_ULDS):
    uld_type = random.choice(ULD_TYPES)
    capacity = ULD_TYPE_CAPACITIES[uld_type]
    
    ulds.append({
        'uld_id': f"ULD{str(i+1).zfill(6)}",
        'uld_type': uld_type,
        'max_weight_kg': capacity['max_weight'],
        'max_volume_cubic_m': capacity['max_volume'],
        'purchase_date': fake.date_between(start_date=datetime(2020, 1, 1), end_date=END_DATE),
        'current_status': random.choices(['In Use', 'Available', 'Under Maintenance', 'Retired'], 
                                        weights=[0.7, 0.15, 0.1, 0.05])[0],
        'current_location': random.choice(AIRPORTS),
        'last_inspection_date': fake.date_between(start_date=datetime(2023, 6, 1), end_date=END_DATE),
        'total_flights': np.random.poisson(lam=50)
    })

df_ulds = pd.DataFrame(ulds)

# ======================
# 3. GENERATE FLIGHTS DIMENSION
# ======================
print("Generating flights dimension...")
flights = []
flight_id_counter = 1

# Create realistic QR cargo routes (DOH-centric)
common_routes = [
    ('DOH', 'LHR'), ('DOH', 'HKG'), ('DOH', 'JFK'), ('DOH', 'NRT'),
    ('DOH', 'SIN'), ('LHR', 'JFK'), ('HKG', 'SIN'), ('FRA', 'DOH')
]

for i in range(NUM_FLIGHTS):
    # 70% of flights involve DOH
    if random.random() < 0.7:
        if random.random() < 0.5:
            origin = 'DOH'
            destination = random.choice([a for a in AIRPORTS if a != 'DOH'])
        else:
            origin = random.choice([a for a in AIRPORTS if a != 'DOH'])
            destination = 'DOH'
    else:
        route = random.choice(common_routes)
        origin, destination = route
    
    flight_date = fake.date_between(start_date=START_DATE, end_date=END_DATE)
    scheduled_departure = datetime.combine(flight_date, datetime.strptime(f"{random.randint(0,23):02d}:{random.randint(0,59):02d}", "%H:%M").time())
    
    # Generate delays (normally distributed around 0 with some outliers)
    departure_delay = np.random.exponential(scale=30) if random.random() < 0.2 else np.random.normal(0, 10)
    departure_delay = max(-15, min(departure_delay, 240))  # Cap between -15 and 240 minutes
    
    flight_duration = random.randint(4, 16) * 60  # 4-16 hours in minutes
    arrival_delay = departure_delay + np.random.normal(0, 15)
    
    actual_departure = scheduled_departure + timedelta(minutes=departure_delay)
    scheduled_arrival = scheduled_departure + timedelta(minutes=flight_duration)
    actual_arrival = scheduled_arrival + timedelta(minutes=arrival_delay)
    
    flights.append({
        'flight_id': f"QR{flight_id_counter:04d}F",
        'flight_number': f"QR{flight_id_counter:04d}",
        'origin_airport': origin,
        'destination_airport': destination,
        'scheduled_departure': scheduled_departure,
        'actual_departure': actual_departure,
        'scheduled_arrival': scheduled_arrival,
        'actual_arrival': actual_arrival,
        'aircraft_type': random.choice(AIRCRAFT_TYPES),
        'planned_payload_kg': np.random.uniform(80000, 120000),
        'actual_payload_kg': None,  # Will be calculated later
        'fuel_consumption_kg': np.random.normal(60000, 5000),
        'flight_status': random.choices(['Completed', 'Diverted', 'Cancelled'], 
                                       weights=[0.95, 0.03, 0.02])[0],
        'captain_id': fake.uuid4()[:8]
    })
    flight_id_counter += 1

df_flights = pd.DataFrame(flights)

# ======================
# 4. GENERATE SHIPMENTS FACT (Main Table)
# ======================
print("Generating shipments fact table...")
shipments = []

# Get active ULDs and customers
active_ulds = df_ulds[df_ulds['current_status'].isin(['In Use', 'Available'])]['uld_id'].tolist()
customer_ids = df_customers['customer_id'].tolist()

for i in range(NUM_SHIPMENTS):
    shipment_id = f"SH{str(i+1).zfill(8)}"
    customer_id = random.choice(customer_ids)
    
    # Get customer segment for bias
    customer_segment = df_customers[df_customers['customer_id'] == customer_id]['customer_segment'].values[0]
    
    # Generate origin and destination
    origin = random.choice(AIRPORTS)
    destination = random.choice([a for a in AIRPORTS if a != origin])
    
    # Booked weight and volume (with some overbooking)
    booked_weight = np.random.exponential(scale=500)
    booked_volume = np.random.exponential(scale=2)
    
    # Actual weight/volume (usually close to booked, but sometimes different)
    if random.random() < 0.1:  # 10% have significant variance
        actual_weight = booked_weight * np.random.uniform(0.7, 1.3)
        actual_volume = booked_volume * np.random.uniform(0.8, 1.2)
    else:
        actual_weight = booked_weight * np.random.uniform(0.95, 1.05)
        actual_volume = booked_volume * np.random.uniform(0.95, 1.05)
    
    # Cargo category based on customer segment
    if customer_segment == 'Pharma Company':
        category = 'Pharma'
    elif customer_segment == 'Perishables Exporter':
        category = 'Perishables'
    elif customer_segment == 'E-commerce Platform':
        category = 'E-commerce'
    else:
        category = random.choices(CARGO_CATEGORIES, weights=[0.3, 0.2, 0.1, 0.1, 0.1, 0.05, 0.1, 0.05])[0]
    
    # Special handling flags
    special_handling = []
    if category in ['Pharma', 'Perishables']:
        special_handling.append('Temperature Controlled')
    if category == 'Valuables':
        special_handling.append('High Security')
    if category == 'Dangerous Goods':
        special_handling.append('HAZMAT')
    
    special_handling = ', '.join(special_handling) if special_handling else 'Standard'
    
    # Timestamps
    booking_date = fake.date_time_between(start_date=START_DATE, end_date=END_DATE)
    
    # Add lead time (days between booking and receiving)
    lead_time_days = np.random.exponential(scale=3)
    receive_date = booking_date + timedelta(days=min(lead_time_days, 14))
    
    # Select a flight for this shipment (match origin/destination)
    possible_flights = df_flights[
        (df_flights['origin_airport'] == origin) & 
        (df_flights['destination_airport'] == destination) &
        (df_flights['scheduled_departure'] > receive_date)
    ]
    
    if len(possible_flights) > 0:
        flight = possible_flights.sample(1).iloc[0]
        flight_id = flight['flight_id']
        scheduled_departure = flight['scheduled_departure']
        actual_departure = flight['actual_departure']
        scheduled_arrival = flight['scheduled_arrival']
        actual_arrival = flight['actual_arrival']
        
        # Calculate if shipment missed flight
        missed_flight = receive_date > scheduled_departure
        
        if missed_flight:
            # Find next available flight
            later_flights = df_flights[
                (df_flights['origin_airport'] == origin) & 
                (df_flights['destination_airport'] == destination) &
                (df_flights['scheduled_departure'] > receive_date)
            ]
            if len(later_flights) > 0:
                flight = later_flights.sample(1).iloc[0]
                flight_id = flight['flight_id']
                scheduled_departure = flight['scheduled_departure']
                actual_departure = flight['actual_departure']
                scheduled_arrival = flight['scheduled_arrival']
                actual_arrival = flight['actual_arrival']
    else:
        # Create a dummy flight entry
        flight_id = None
        scheduled_departure = receive_date + timedelta(hours=np.random.uniform(12, 48))
        actual_departure = scheduled_departure + timedelta(minutes=np.random.normal(30, 20))
        scheduled_arrival = scheduled_departure + timedelta(hours=np.random.uniform(4, 16))
        actual_arrival = scheduled_arrival + timedelta(minutes=np.random.normal(15, 30))
    
    # Select ULD (sometimes null for loose cargo)
    if random.random() < 0.8:  # 80% use ULD
        uld_id = random.choice(active_ulds) if active_ulds else None
    else:
        uld_id = None
    
    # Delivery dates (with delays)
    transit_days = np.random.exponential(scale=2)
    delivery_date = actual_arrival + timedelta(days=transit_days)
    
    # Customs clearance delay (20% of shipments)
    if random.random() < 0.2:
        delivery_date += timedelta(days=np.random.exponential(scale=1))
    
    # Revenue calculation
    base_rate = {
        'General Cargo': 3.5,
        'Perishables': 4.5,
        'Pharma': 6.0,
        'Valuables': 8.0,
        'Dangerous Goods': 5.0,
        'Live Animals': 7.0,
        'E-commerce': 3.0,
        'Express': 9.0
    }.get(category, 3.5)
    
    # Add premium for special handling
    if 'Temperature Controlled' in special_handling:
        base_rate *= 1.3
    if 'High Security' in special_handling:
        base_rate *= 1.5
    
    revenue_usd = actual_weight * base_rate
    
    # Determine status
    current_time = END_DATE
    if delivery_date < current_time:
        status = 'Delivered'
    elif actual_arrival < current_time:
        status = 'Arrived'
    elif actual_departure < current_time:
        status = 'Departed'
    elif receive_date < current_time:
        status = 'Received'
    else:
        status = 'Booked'
    
    shipments.append({
        'shipment_id': shipment_id,
        'customer_id': customer_id,
        'uld_id': uld_id,
        'flight_id': flight_id,
        'origin_airport': origin,
        'destination_airport': destination,
        'cargo_category': category,
        'special_handling': special_handling,
        'booked_weight_kg': round(booked_weight, 2),
        'actual_weight_kg': round(actual_weight, 2),
        'booked_volume_cubic_m': round(booked_volume, 2),
        'actual_volume_cubic_m': round(actual_volume, 2),
        'declared_value_usd': np.random.lognormal(mean=9, sigma=1.5),
        'timestamp_booking': booking_date,
        'timestamp_received': receive_date,
        'timestamp_departure': actual_departure if flight_id else None,
        'timestamp_arrival': actual_arrival if flight_id else None,
        'timestamp_delivery': delivery_date if status == 'Delivered' else None,
        'revenue_usd': round(revenue_usd, 2),
        'fuel_surcharge_usd': round(revenue_usd * 0.15, 2),
        'security_surcharge_usd': round(revenue_usd * 0.05, 2),
        'total_charges_usd': round(revenue_usd * 1.2, 2),
        'status': status,
        'priority': random.choices(['Normal', 'Priority', 'Express'], weights=[0.7, 0.2, 0.1])[0],
        'insurance_required': random.random() < 0.3,
        'customs_cleared': status in ['Arrived', 'Delivered'] and random.random() < 0.85,
        'damage_flag': random.random() < 0.01,
        'temperature_violation_flag': category in ['Pharma', 'Perishables'] and random.random() < 0.02
    })

df_shipments = pd.DataFrame(shipments)

# ======================
# 5. GENERATE DAILY OPERATIONS TABLE
# ======================
print("Generating daily operations table...")
daily_ops = []
current_date = START_DATE

while current_date <= END_DATE:
    # Generate daily metrics
    day_shipments = df_shipments[
        (df_shipments['timestamp_booking'].dt.date == current_date.date())
    ]
    
    day_revenue = day_shipments['total_charges_usd'].sum()
    day_shipment_count = len(day_shipments)
    
    # Simulate operational issues
    operational_issues = []
    if random.random() < 0.1:  # 10% chance of issues
        if random.random() < 0.5:
            operational_issues.append('Equipment Shortage')
        if random.random() < 0.3:
            operational_issues.append('Staff Shortage')
        if random.random() < 0.2:
            operational_issues.append('Weather Delay')
    
    daily_ops.append({
        'date': current_date.date(),
        'total_shipments': day_shipment_count,
        'total_revenue_usd': round(day_revenue, 2),
        'total_weight_kg': day_shipments['actual_weight_kg'].sum(),
        'operational_issues': ', '.join(operational_issues) if operational_issues else 'None',
        'staff_count': np.random.poisson(lam=50),
        'equipment_utilization_rate': np.random.beta(a=2, b=2),
        'weather_condition': random.choice(['Clear', 'Rain', 'Fog', 'Storm']),
        'temperature_celsius': np.random.normal(25, 5)
    })
    
    current_date += timedelta(days=1)

df_daily_ops = pd.DataFrame(daily_ops)

# ======================
# 6. GENERATE FLIGHT LOAD FACT TABLE
# ======================
print("Generating flight load fact table...")
flight_loads = []

for _, flight in df_flights.iterrows():
    flight_shipments = df_shipments[df_shipments['flight_id'] == flight['flight_id']]
    
    if len(flight_shipments) > 0:
        total_weight = flight_shipments['actual_weight_kg'].sum()
        total_volume = flight_shipments['actual_volume_cubic_m'].sum()
        
        # Get ULDs used on this flight
        ulds_on_flight = flight_shipments['uld_id'].dropna().unique()
        uld_count = len(ulds_on_flight)
        
        # Calculate utilization
        aircraft_capacity = 100000  # Standard widebody freighter
        weight_utilization = (total_weight / aircraft_capacity) * 100
        
        flight_loads.append({
            'flight_id': flight['flight_id'],
            'flight_date': flight['scheduled_departure'].date(),
            'total_payload_kg': round(total_weight, 2),
            'total_volume_cubic_m': round(total_volume, 2),
            'uld_count': uld_count,
            'shipment_count': len(flight_shipments),
            'weight_utilization_percent': round(weight_utilization, 2),
            'revenue_per_flight_usd': flight_shipments['total_charges_usd'].sum(),
            'fuel_efficiency_kg_per_ton': np.random.normal(450, 50),
            'co2_emissions_kg': total_weight * np.random.normal(3, 0.5)
        })

df_flight_loads = pd.DataFrame(flight_loads)

# Update flights with actual payload
for _, load in df_flight_loads.iterrows():
    df_flights.loc[df_flights['flight_id'] == load['flight_id'], 'actual_payload_kg'] = load['total_payload_kg']

# ======================
# 7. SAVE TO CSV FILES
# ======================
print("Saving CSV files...")

# Create filenames
files_to_save = {
    'customers_dim.csv': df_customers,
    'ulds_dim.csv': df_ulds,
    'flights_dim.csv': df_flights,
    'shipments_fact.csv': df_shipments,
    'daily_operations.csv': df_daily_ops,
    'flight_loads_fact.csv': df_flight_loads
}

for filename, df in files_to_save.items():
    df.to_csv(filename, index=False)
    print(f"  Saved {filename} with {len(df)} rows")

# ======================
# 8. CREATE SAMPLE QUERIES FILE
# ======================
print("\nGenerating sample SQL queries...")

sample_queries = """
-- SAMPLE SQL QUERIES FOR QATAR AIRWAYS CARGO ANALYSIS PROJECT

-- 1. Get daily shipment counts and revenue
SELECT 
    DATE(timestamp_booking) as booking_date,
    COUNT(*) as shipment_count,
    SUM(total_charges_usd) as daily_revenue,
    AVG(total_charges_usd) as avg_shipment_value
FROM shipments_fact
GROUP BY DATE(timestamp_booking)
ORDER BY booking_date DESC;

-- 2. Top performing routes by revenue
SELECT 
    origin_airport,
    destination_airport,
    COUNT(*) as shipment_count,
    SUM(total_charges_usd) as total_revenue,
    AVG(total_charges_usd) as avg_revenue_per_shipment
FROM shipments_fact
GROUP BY origin_airport, destination_airport
ORDER BY total_revenue DESC
LIMIT 10;

-- 3. ULD utilization analysis
SELECT 
    u.uld_type,
    COUNT(DISTINCT s.shipment_id) as shipments_assigned,
    AVG(s.actual_weight_kg / u.max_weight_kg) * 100 as avg_weight_utilization_percent,
    AVG(s.actual_volume_cubic_m / u.max_volume_cubic_m) * 100 as avg_volume_utilization_percent
FROM ulds_dim u
LEFT JOIN shipments_fact s ON u.uld_id = s.uld_id
WHERE s.uld_id IS NOT NULL
GROUP BY u.uld_type
ORDER BY shipments_assigned DESC;

-- 4. On-time performance by flight
SELECT 
    f.flight_number,
    f.origin_airport,
    f.destination_airport,
    AVG(TIMESTAMPDIFF(MINUTE, f.scheduled_departure, f.actual_departure)) as avg_departure_delay_min,
    AVG(TIMESTAMPDIFF(MINUTE, f.scheduled_arrival, f.actual_arrival)) as avg_arrival_delay_min,
    COUNT(s.shipment_id) as total_shipments
FROM flights_dim f
LEFT JOIN shipments_fact s ON f.flight_id = s.flight_id
GROUP BY f.flight_id, f.flight_number, f.origin_airport, f.destination_airport
HAVING total_shipments > 0;

-- 5. Customer segmentation analysis
SELECT 
    c.customer_segment,
    c.contract_type,
    COUNT(DISTINCT s.shipment_id) as total_shipments,
    SUM(s.total_charges_usd) as total_revenue,
    AVG(s.total_charges_usd) as avg_shipment_value,
    COUNT(DISTINCT c.customer_id) as customer_count
FROM customers_dim c
JOIN shipments_fact s ON c.customer_id = s.customer_id
GROUP BY c.customer_segment, c.contract_type
ORDER BY total_revenue DESC;

-- 6. Cargo category performance
SELECT 
    cargo_category,
    COUNT(*) as shipment_count,
    SUM(actual_weight_kg) as total_weight_kg,
    SUM(total_charges_usd) as total_revenue,
    AVG(total_charges_usd / actual_weight_kg) as rate_per_kg
FROM shipments_fact
GROUP BY cargo_category
ORDER BY total_revenue DESC;

-- 7. Delay analysis
SELECT 
    origin_airport,
    AVG(TIMESTAMPDIFF(HOUR, timestamp_received, timestamp_departure)) as avg_ground_time_hrs,
    AVG(TIMESTAMPDIFF(HOUR, timestamp_departure, timestamp_arrival)) as avg_transit_time_hrs,
    AVG(TIMESTAMPDIFF(HOUR, timestamp_arrival, timestamp_delivery)) as avg_delivery_time_hrs,
    COUNT(*) as shipment_count
FROM shipments_fact
WHERE timestamp_delivery IS NOT NULL
GROUP BY origin_airport
ORDER BY avg_ground_time_hrs DESC;
"""

with open('sample_sql_queries.sql', 'w') as f:
    f.write(sample_queries)
print("  Saved sample_sql_queries.sql")

# ======================
# 9. CREATE DATA DICTIONARY
# ======================
data_dictionary = """
DATA DICTIONARY - Qatar Airways Cargo Analytics

1. customers_dim.csv
   - customer_id: Unique customer identifier
   - customer_name: Company name
   - customer_segment: Business segment (Freight Forwarder, Direct Shipper, etc.)
   - contract_type: Type of contract with QR Cargo
   - credit_rating: Customer credit rating (A=Best, D=Worst)
   - yearly_spend_usd: Estimated annual spend

2. ulds_dim.csv
   - uld_id: Unit Load Device identifier
   - uld_type: Type of ULD (AKE, AKH, etc.)
   - max_weight_kg: Maximum weight capacity
   - max_volume_cubic_m: Maximum volume capacity
   - current_status: Operational status
   - current_location: Current airport location

3. flights_dim.csv
   - flight_id: Unique flight identifier
   - flight_number: QR flight number
   - origin/destination_airport: Route endpoints
   - scheduled/actual_departure/arrival: Timestamps
   - aircraft_type: Type of aircraft
   - flight_status: Completed, Diverted, Cancelled

4. shipments_fact.csv (MAIN TABLE)
   - shipment_id: Unique shipment identifier
   - customer_id: Reference to customers_dim
   - uld_id: Reference to ulds_dim (nullable)
   - flight_id: Reference to flights_dim (nullable)
   - origin/destination_airport: Shipment route
   - cargo_category: Type of cargo
   - booked/actual_weight_kg: Planned vs actual weight
   - booked/actual_volume_cubic_m: Planned vs actual volume
   - timestamp_booking/received/departure/arrival/delivery: Shipment milestones
   - total_charges_usd: Total revenue from shipment
   - status: Current shipment status

5. daily_operations.csv
   - date: Operation date
   - total_shipments: Shipments handled that day
   - total_revenue_usd: Daily revenue
   - operational_issues: Any operational challenges
   - weather_condition: Weather at hub

6. flight_loads_fact.csv
   - flight_id: Reference to flights_dim
   - total_payload_kg: Total weight carried
   - weight_utilization_percent: % of aircraft capacity used
   - revenue_per_flight_usd: Revenue generated by flight
   - fuel_efficiency_kg_per_ton: Fuel efficiency metric
"""

with open('data_dictionary.md', 'w') as f:
    f.write(data_dictionary)
print("  Saved data_dictionary.md")

# ======================
# 10. SUMMARY STATISTICS
# ======================
print("\n" + "="*50)
print("DATASET GENERATION COMPLETE")
print("="*50)
print(f"• Customers: {len(df_customers)}")
print(f"• ULDs: {len(df_ulds)}")
print(f"• Flights: {len(df_flights)}")
print(f"• Shipments: {len(df_shipments)}")
print(f"• Daily Operations Records: {len(df_daily_ops)}")
print(f"• Flight Load Records: {len(df_flight_loads)}")
print("\nTotal Revenue in Dataset: ${:,.2f}".format(df_shipments['total_charges_usd'].sum()))
print("Avg Shipment Value: ${:,.2f}".format(df_shipments['total_charges_usd'].mean()))
print("Date Range: {} to {}".format(START_DATE.date(), END_DATE.date()))

# Calculate some KPIs
total_weight = df_shipments['actual_weight_kg'].sum()
print(f"Total Cargo Weight: {total_weight:,.0f} kg")

# On-time delivery rate (assuming delivered within 2 days of scheduled arrival is on-time)
if 'timestamp_delivery' in df_shipments.columns and 'timestamp_arrival' in df_shipments.columns:
    delivered = df_shipments[df_shipments['status'] == 'Delivered']
    if len(delivered) > 0:
        avg_delivery_time = (delivered['timestamp_delivery'] - delivered['timestamp_arrival']).dt.days.mean()
        print(f"Avg Delivery Time After Arrival: {avg_delivery_time:.1f} days")

print("\nFiles created:")
print("1. customers_dim.csv")
print("2. ulds_dim.csv")
print("3. flights_dim.csv")
print("4. shipments_fact.csv")
print("5. daily_operations.csv")
print("6. flight_loads_fact.csv")
print("7. sample_sql_queries.sql")
print("8. data_dictionary.md")

Generating customer dimension...
Generating ULD master...
Generating flights dimension...
Generating shipments fact table...
Generating daily operations table...
Generating flight load fact table...
Saving CSV files...
  Saved customers_dim.csv with 500 rows
  Saved ulds_dim.csv with 5000 rows
  Saved flights_dim.csv with 2000 rows
  Saved shipments_fact.csv with 100000 rows
  Saved daily_operations.csv with 397 rows
  Saved flight_loads_fact.csv with 1617 rows

Generating sample SQL queries...
  Saved sample_sql_queries.sql
  Saved data_dictionary.md

DATASET GENERATION COMPLETE
• Customers: 500
• ULDs: 5000
• Flights: 2000
• Shipments: 100000
• Daily Operations Records: 397
• Flight Load Records: 1617

Total Revenue in Dataset: $341,497,097.84
Avg Shipment Value: $3,414.97
Date Range: 2024-01-01 to 2025-01-31
Total Cargo Weight: 50,037,272 kg
Avg Delivery Time After Arrival: 1.7 days

Files created:
1. customers_dim.csv
2. ulds_dim.csv
3. flights_dim.csv
4. shipments_fact.csv
5. dail